<a href="https://colab.research.google.com/github/glf58/Projet_Covid/blob/GLF/generate_dataset_for_ffd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objectif
Ce notebook sert a generer un dataset dans un format compatible avec la methode flow_from_directory. Voir la definition de la fonction generate_and_split_data.
On commence par lire les donnees dans l'archive, puis, une fois les images distribuees, on les redistribue dans les bons repertoires. On peut egalement creer et sauver une archive (.zip) de ces donnees qu'il suffira d'aller lire pour entrainer le modele.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time
import shutil
import datetime 

from skimage import exposure

from google.colab import drive
from zipfile import ZipFile

from tensorflow.keras.utils import load_img, img_to_array, save_img

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
archive='/content/drive/MyDrive/kaggle/covid19-radiography-database.zip'
with ZipFile(archive, 'r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
path_data='/content/COVID-19_Radiography_Dataset' #repertoire ou on dezippe les images initiales
path_out = '/content/C19' #repertoire ou on va copier les images qu'on voudra zipper
save_archive = True       #True pour sauver l'archive dans le drive perso, False sinon; dans ce cas les donnees seront perdues apres la session de colab
histo_equalize = True     #True pour renormaliser les expositions, False pour garder les images d'origine
if histo_equalize:
  he='histo_equal'
else:
  he=''
filename = '/content/drive/MyDrive/kaggle/covid19-'+he+'-dataset_for_ffd'  #chemin de destination de l'archive que je sauve dans mon drive personnel afin de ne pas le perdre.
data_types=['Test', 'Validation', 'Train']
cats = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']
initial_size = (299,299)  #taille initiale des images
target_size = (299,299)   #redimensionnement des images
Nmax = 11200              #nb max d'images par categorie a considerer
test_size = 15            #pourcentage images dans le repertoire de test: 15 pour 15%
val_size = 15             #pourcentage images dans le repertoire de validation: 15 pour 15%

In [ ]:
def generate_and_split_data(test_size, val_size, cats, data_types, Nmax, histo_equalize, path_in, path_out, save_archive, filename):
  ### cette fonction va lire les images dans 'path_in' et creer une arborescence compatible avec la methode flow_from_directory dans le repertoire 'path_out'.
  ### Chaque image sera lue dans le repertoire 'path_in' et collee dans le bon repertoire cible de 'path_out'. On considere que les premieres images iront 
  ### dans le repertoire de 'Test' et les autres dans le repertoire 'Train'. Pour chaque categorie, la somme du nombre d'images transferee sur Test et Train est limite a Nmax.


  #on detruit les data precedentes
  shutil.rmtree(path_out, ignore_errors=True)

  #on cree les repertoires avec la bonne arborescence
  os.mkdir(path_out)
  for data_type in data_types:
    os.mkdir(os.path.join(path_out, data_type))
    for cat in cats:
        os.mkdir(os.path.join(path_out, data_type, cat))

  #plt.figure()
  deb = time.time()
  cpteur = 1
  freq_affiche = 500
  for cat in cats:
    n_data_available = len(os.listdir(os.path.join(path_in, cat, 'images')))
    n_data = min(Nmax, n_data_available)
    new_idx = np.arange(1,n_data_available)         #on prend tous les indices possibles
    np.random.shuffle(new_idx)                      #on les melange
    new_idx = new_idx[:n_data]                      # on ne prend que les n_data premiers elements
    n_test = test_size * n_data // 100    
    n_val = val_size * n_data // 100    
    print("Pour la categorie {}, on va copier {} images (parmi {} disponibles) dont {} dans l'ensemble Test et {} dans l'ensemble validation".format(cat, n_data, n_data_available, n_test, n_val))
    for i, idx in enumerate(new_idx):
      source = os.path.join(path_in, cat, 'images', cat)+'-'+str(idx+1)+'.png'
      #on definit le repertoire d'arrivee
      if i < n_test:
        dest = os.path.join(path_out, data_types[0], cat, cat)+'-'+str(idx+1)+'.png' #on garde le meme nom d'image
      elif i < n_val + n_test:
        dest = os.path.join(path_out, data_types[1], cat, cat)+'-'+str(idx+1)+'.png' #on garde le meme nom d'image
      else:
        dest = os.path.join(path_out, data_types[2], cat, cat)+'-'+str(idx+1)+'.png' #on garde le meme nom d'image

      if histo_equalize:
        img = load_img(source, color_mode='grayscale')
        #plt.subplot(121)
        #plt.imshow(img, cmap='gray')
        array = img_to_array(img, dtype='uint16').reshape(target_size)
        new_img = exposure.equalize_adapthist(array, clip_limit=0.03)
        #plt.subplot(122)
        #plt.imshow(new_img, cmap='gray')
        #plt.show()
        save_img(dest, new_img.reshape([target_size[0], target_size[1], 1]))
#        img = cv2.imread(source, cv2.IMREAD_GRAYSCALE)
#        print(img.shape, img.min(), img.max())
#        new_img = exposure.equalize_adapthist(img, clip_limit=0.03)
#        imsave(dest, img_as_ubyte(new_img))
      else:        
        shutil.copy(source,dest)

      if cpteur%freq_affiche == 0:
        print("{} images copiees en {} secondes. {} images copiees a ce stade".format(freq_affiche, time.time() - deb, cpteur))
        deb = time.time()
      cpteur += 1
 
  if save_archive:
    shutil.make_archive(base_name=filename, format="zip", root_dir=path_out)


In [ ]:
generate_and_split_data(test_size, val_size, cats, data_types, Nmax, histo_equalize, path_data, path_out, save_archive, filename)

Pour la categorie COVID, on va copier 3616 images (parmi 3616 disponibles) dont 542 dans l'ensemble Test et 542 dans l'ensemble validation
500 images copiees en 15.289548873901367 secondes. 500 images copiees a ce stade
500 images copiees en 15.280012130737305 secondes. 1000 images copiees a ce stade
500 images copiees en 15.319929838180542 secondes. 1500 images copiees a ce stade
500 images copiees en 15.023252248764038 secondes. 2000 images copiees a ce stade
500 images copiees en 15.595375299453735 secondes. 2500 images copiees a ce stade
500 images copiees en 15.020164489746094 secondes. 3000 images copiees a ce stade
500 images copiees en 15.223627805709839 secondes. 3500 images copiees a ce stade
Pour la categorie Lung_Opacity, on va copier 6012 images (parmi 6012 disponibles) dont 901 dans l'ensemble Test et 901 dans l'ensemble validation
500 images copiees en 15.118233680725098 secondes. 4000 images copiees a ce stade
500 images copiees en 15.064002275466919 secondes. 4500 imag

In [ ]:
tot_img = 0
for cat in cats:
  n_data_available = len(os.listdir(os.path.join(path_data, cat, 'images')))
  tot_img += n_data_available
  print('dans le repertoire de base, la categorie {} possede {} images'.format(cat, n_data_available))
print("total images disponibles: ", tot_img)

dans le repertoire de base, la categorie COVID possede 3616 images
dans le repertoire de base, la categorie Lung_Opacity possede 6012 images
dans le repertoire de base, la categorie Normal possede 10192 images
dans le repertoire de base, la categorie Viral Pneumonia possede 1345 images
total images disponibles:  21165


In [ ]:
for data_type in data_types:
  print(data_type)
  n_type = 0
  for cat in cats:    
    print('la categorie {} possede {} images'.format(cat,len(os.listdir(os.path.join(path_out, data_type, cat)))))
    n_type += len(os.listdir(os.path.join(path_out, data_type, cat)))
  print('les donnees de {} comportent {} images'.format(data_type, n_type))

Test
la categorie COVID possede 542 images
la categorie Lung_Opacity possede 901 images
la categorie Normal possede 1528 images
la categorie Viral Pneumonia possede 201 images
les donnees de Test comportent 3172 images
Validation
la categorie COVID possede 542 images
la categorie Lung_Opacity possede 901 images
la categorie Normal possede 1528 images
la categorie Viral Pneumonia possede 201 images
les donnees de Validation comportent 3172 images
Train
la categorie COVID possede 2531 images
la categorie Lung_Opacity possede 4209 images
la categorie Normal possede 7135 images
la categorie Viral Pneumonia possede 942 images
les donnees de Train comportent 14817 images


In [ ]:
#verification
os.mkdir('/content/covid19-dataset_for_ffd/')
with ZipFile(filename+".zip", 'r') as zip_ref:
  zip_ref.extractall(path='/content/covid19-dataset_for_ffd/')
  

FileExistsError: ignored

In [ ]:
a = np.array([1,2,3,4,5,6])
np.random.shuffle(a)
print(a)

b = np.arange(1,100)
print(b)
np.random.shuffle(b)
print(b)

[5 2 1 6 4 3]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98 99]
[37 32 22 20 90 13  7 40 70 43 41 62  3 93 11 50 39  4 84 16 21 18  1 27
 66 98 73 35 79 24 78 72 61 52 81 31 48 77 96 10 57 82 29 25 56 17  5 75
  9 49 65 59 83 86 55 19 33 53 88 92 63 67 46 38 69 64 28 45 80 34  8 89
 30 91 47 97 87 44 54 95 36  2 12 74 85 26 42  6 23 14 99 58 71 76 15 94
 68 51 60]


In [ ]:
    new_idx1 = np.arange(1,10000)         #on prend tous les indices possibles
    np.random.shuffle(new_idx1)                      #on les melange
    new_idx1 = new_idx1[:100]                      # on ne prend que les n_data premiers elements
    print(new_idx1)

[1887 7454 7584 7529 7322 3689 4244 6449  357 6585 3559  826 3055 1357
 6036 2453 4487  430 2189 9593 9679  457 5634 2335 2411 5278 5801 9823
 3780 7946 2095 8773 3430 4407 5860 8187  757 4209 7395 5312  713 6551
 9791 6912  538  465 4694 2496 4397 2978 3032 1769 2150 1503 5579   46
 6886 1773 3378  235 7561 4841 8810 7562 8023  152 8397 4380 5565 5756
 9965 6549 5918 4777 8298 5813 1756 9556 8297 4224 3343 4998 2000 1730
 1359 8442 2254 8924 3907 3461 9192 4365  291 3646 9193 7767  466 3751
 3040 3070]


In [ ]:
#avec randint, on peut avoir 2 fois le meme indice !! ne pas utiliser
a = np.random.randint(1,100,100)
print(a)

[81 35 16 89 23 90 85 44 34 40 68 95 50 26 37 26 83 92 94 76 78 20 68  6
 54 84 20  8  3 27 87 25 14 53 47  1 70 57  2 35 87 75 86 78 62 17 22 10
 51 75 81 32 83 99 20 16 18 82 83 94 64 13 64 46 37 11 84 27 50 74 66 24
 45 93 88 14  7 53 98 63 52  3 35 87 60 69 25 33 64 38 89 43 70 39 47 55
 73 79 75 39]
